In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath("")))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
# Get multiple tags conveniently
marker_tag_names = [
    "AI",
    "Marker Status: Confirmed",
    "Marker Source: PySceneDetect",
    "Marker Source: Skier AI",
    "Marker Source: Manual", 
    "Marker Source: timestamp.trade",
    "Marker Source: TPDB"
]

marker_tags = stash_client.get_tags_by_names(marker_tag_names)

# Confirmed markers with unknown source

In [ ]:
confirmed_markers_with_unknown_source = stash.find_scene_markers(
    {
        "tags": {
            "value": [marker_tags.marker_status_confirmed["id"]],
            "modifier": "INCLUDES_ALL",
            "excludes": [marker_tags.marker_source_manual["id"], marker_tags.marker_source_timestamp_trade["id"], marker_tags.marker_source_tpdb["id"], marker_tags.marker_source_skier_ai["id"]],
        }
    },
    fragment="id primary_tag { id name } tags { id name } scene { id title }",
)
confirmed_markers_with_unknown_source

In [ ]:
for confirmed_marker in confirmed_markers_with_unknown_source:
    existing_tags = confirmed_marker["tags"]
    existing_tags_ids = [tag["id"] for tag in existing_tags]
    updated_tags_ids = existing_tags_ids + [marker_tags.marker_source_skier_ai["id"]]
    print(confirmed_marker["scene"]["title"])
    print(updated_tags_ids)
    stash.update_scene_marker({
        "id": confirmed_marker["id"],
        "tag_ids": updated_tags_ids
    })

# Markers with AI tags with unknown source

In [ ]:
ai_markers_with_unknown_source = stash.find_scene_markers(
    {
        "tags": {
            "value": [marker_tags.ai["id"]],
            "depth": 1,
            "modifier": "INCLUDES_ALL",
            "excludes": [
                marker_tags.marker_status_confirmed["id"],
                marker_tags.marker_source_manual["id"],
                marker_tags.marker_source_timestamp_trade["id"],
                marker_tags.marker_source_tpdb["id"],
                marker_tags.marker_source_pyscenedetect["id"],
                marker_tags.marker_source_manual["id"],
                marker_tags.marker_source_skier_ai["id"],
            ],
        }
    },
    filter = {"per_page": 10000},
    fragment="id primary_tag { id name } tags { id name } scene { id title }",
)
ai_markers_with_unknown_source[0:5]

In [ ]:
for marker in ai_markers_with_unknown_source:
    stash_client.update_marker_tags(marker, [marker_tags.marker_source_skier_ai["id"]], [])

# Unconfirm markers

In [ ]:
confirmed_markers_in_a_scene = stash.find_scene_markers(
    {
        "scene_filter": {
            "tags": {
                "value": [marker_tags.marker_status_confirmed["id"]],
                "modifier": "INCLUDES_ALL",
            }
        },
        "tags": {
            "value": [marker_tags.marker_status_confirmed["id"]],
            "modifier": "INCLUDES_ALL",
        }
    },
    fragment="id primary_tag { id name } tags { id name } scene { id title }",
)
confirmed_markers_in_a_scene

In [ ]:
for confirmed_marker in confirmed_markers_in_a_scene:
    stash_client.update_marker_tags(confirmed_marker, [], [marker_tags.marker_status_confirmed["id"]])